In [1]:
"""
Connect Aliyun and commit MySQL
"""
 
import requests
import json
 
 
def aliq(db, req):
    url = 'http://localhost:8008/aliq?sql=%s&db=%s' % (requests.utils.quote(req), db)
    resp = requests.get(url)
    res = resp.content.decode()
    js = json.loads(res)
    return js

In [2]:
"""
Connect senior:Mainland
"""
import pandas as pd
import json
import requests
def ssql(sql):
    admin_token = '2cebeb5c7edac5ef0d7c5a640e69fc7d43bf2cd6a24ce43dfee3dd33662c0bab'
    url = 'http://sensor.wb-intra.com/api/sql/query?token=%s&project=production' % admin_token
    data = {'q': sql, 'format': 'json'}
    req = requests.post(url,data)
    req_dec = req.content.decode()
    try:
        req_json = json.loads('[' + req_dec.replace('\n', ',')[:-1] + ']')
        df_d_id = pd.DataFrame(req_json)
        return df_d_id
    except:
        print(req_dec)

In [5]:
df_wolf = ssql("""
    SELECT t1.distinct_id
FROM
  (SELECT date, distinct_id
   FROM events
   WHERE event = 'register'
     AND appId IN ('20014',
                   '30015')
     AND date = current_date() - interval 1 DAY
     AND hour(time) BETWEEN 9 AND 11
   GROUP BY 1,
            2) t1
LEFT JOIN
  (SELECT date, distinct_id
   FROM events
   WHERE event = 'gameOver'
     AND gameTypeId = 1800
     AND date = current_date() - interval 1 DAY
   GROUP BY 1,
            2)t2 ON t1.distinct_id = t2.distinct_id
AND t1.date = t2.date
WHERE t2.distinct_id IS NULL
GROUP BY 1
""")
wolf_user_list = list(df_wolf.distinct_id)
group_user_dict = {str(i) : [] for i in range(100)}
for i in wolf_user_list:
    no = str(int(i[-2:]))
    group_user_dict[no].append(i)

all_info = []
for i in range(100):    
    if group_user_dict[str(i)]:
        table = 'user_phone_%s' % str(i)

        user_list = group_user_dict[str(i)]
        if len(user_list) == 1:
            user_list.append('123')
        sql = """
            select uid, phone_number
            from {table}
            where uid in{user_list}
        """.format(table=table, user_list=tuple(user_list))
        
        data = aliq('spy', sql)
        if data[0]:
            all_info.extend(data)
df_phone = pd.DataFrame.from_dict([{'distinct_id': str(i[0]), 'phone_number': str(i[1])} for i in all_info])
df_merge = pd.merge(df_wolf, df_phone, on='distinct_id', how='left')
df_merge

,distinct_id,phone_number
0,123002862,8613984474513
1,131061416,NaN
2,113786450,8617631580965
3,110008313,8615307502873
4,132848878,NaN
...,...,...
84966,120094235,8618086805156
84967,132913713,8618968521211
84968,133094936,8615348204992
84969,147050307,NaN


In [6]:
df_merge.to_csv('./未进行游戏的新注册用户.csv')